## Cleaning Food Inspection (2018 Forward) 
**Project:** Food Inspection: Modeling Risk Assignment Using Ordinal Logistic Regression  
**Authors:** Christopher Hein, Yuming Liao, Andrew McCurdy, Nina Randorf, Melanie Tran

This script cleans the City of Chicago Food Inspections (from 2018 to present) file available on the Chicago Data Portal.  
Date of pull: 19Feb2021. 

The script creates a smaller dataframe composed of the Facility Type, Risk, Inspection Type, and Violations.  Rows are filtered to only reflect "Pass" (inspection result), "Restaurant" (facility type), and "Canvass" (inspection type).  Inspection dates are grouped into seasons (spring, summer, fall, winter) and violations are grouped in health inspection checklist focus areas.  

Link to City of Chicago Data Portal:  
https://data.cityofchicago.org/Health-Human-Services/Food-Inspections-7-1-2018-Present/qizy-d2wf/data

Link to health cinspection checklist:  
https://www.chicago.gov/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/understand_healthcoderequirementsforfoodestablishments.html

In [1]:
#load libraries
import pandas as pd
import regex as re
import numpy as np

In [2]:
#load data
df = pd.read_csv("Food_Inspections_-_7_1_2018_-_Present.csv")
print(df.shape)
df.head()


(43574, 17)


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2184275,MCCB,MCCB,2600357.0,Restaurant,Risk 1 (High),2138 S ARCHER AVE,CHICAGO,IL,60616.0,7/3/2018,License,Pass w/ Conditions,NaN,41.853682,-87.633396,"(-87.63339594961039, 41.853681636731515)"
1,2184211,JAMES KITCHEN + BAR,JAMES HOTEL CHICAGO,1884681.0,Restaurant,Risk 1 (High),616 N RUSH ST,CHICAGO,IL,60611.0,7/3/2018,Canvass,Out of Business,NaN,41.893131,-87.625559,"(-87.62555906428202, 41.89313081087328)"
2,2182184,CAFE EL TAPATIO,CAFE EL TAPATIO,2432567.0,Restaurant,Risk 1 (High),3400-3402 N ASHLAND AVE,CHICAGO,IL,60657.0,7/3/2018,Canvass Re-Inspection,Pass,NaN,41.943432,-87.668913,"(-87.66891275635102, 41.943432313397565)"
3,2170218,ROYS LUNCH BAG,ROYS LUNCH BAG,46653.0,Restaurant,Risk 1 (High),403 E 71ST ST,CHICAGO,IL,60619.0,7/3/2018,Canvass Re-Inspection,Pass,NaN,41.765605,-87.615114,"(-87.61511390884785, 41.76560498384187)"
4,2182196,MCDONALD'S,MCDONALD'S,1840645.0,Restaurant,Risk 2 (Medium),6740 N CLARK ST,CHICAGO,IL,60626.0,7/3/2018,Canvass,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,42.004693,-87.673036,"(-87.67303568688851, 42.004692640610834)"


In [12]:
#initial exploratory analysis
df["Facility"].value_counts()

43574


Restaurant                         29431
Grocery Store                       5549
School                              2385
Children's Services Facility        1337
Daycare Above and Under 2 Years      712
                                   ...  
PHARMACY                               1
Church                                 1
Assisted Living                        1
Coffee shop                            1
GROCERY/SERVICE GAS STATION            1
Name: Facility Type, Length: 258, dtype: int64

In [13]:
df["Inspection Type"].value_counts()

Canvass                                   23061
License                                    6309
Canvass Re-Inspection                      4945
Complaint                                  4124
License Re-Inspection                      1750
Complaint Re-Inspection                    1749
Short Form Complaint                       1058
Non-Inspection                              303
Suspected Food Poisoning                    126
Recent Inspection                           119
Suspected Food Poisoning Re-inspection       22
Consultation                                  5
COVID COMPLAINT                               1
OFFICE ASSIGNMENT                             1
Not Ready                                     1
Name: Inspection Type, dtype: int64

In [14]:
df["Results"].value_counts()

Pass w/ Conditions      16317
Pass                    12058
Fail                     8206
Out of Business          3579
No Entry                 2316
Not Ready                1088
Business Not Located       10
Name: Results, dtype: int64

In [255]:
#overwrite variable with smaller subset 
df = df[df.Results == "Pass"] #reduces rows from 43,574 to 12,058
print(df.shape)
df.head()

(12058, 17)


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
2,2182184,CAFE EL TAPATIO,CAFE EL TAPATIO,2432567.0,Restaurant,Risk 1 (High),3400-3402 N ASHLAND AVE,CHICAGO,IL,60657.0,7/3/2018,Canvass Re-Inspection,Pass,NaN,41.943432,-87.668913,"(-87.66891275635102, 41.943432313397565)"
3,2170218,ROYS LUNCH BAG,ROYS LUNCH BAG,46653.0,Restaurant,Risk 1 (High),403 E 71ST ST,CHICAGO,IL,60619.0,7/3/2018,Canvass Re-Inspection,Pass,NaN,41.765605,-87.615114,"(-87.61511390884785, 41.76560498384187)"
8,2184222,TAQUERIA EL RANCHITO,TAQUERIA EL RANCHITO,60184.0,Restaurant,Risk 1 (High),2829 N MILWAUKEE AVE,CHICAGO,IL,60618.0,7/3/2018,Complaint Re-Inspection,Pass,NaN,41.932657,-87.713056,"(-87.71305610116421, 41.93265693660558)"
10,2184212,IRONSIDE BAR AND GALLEY,IRONSIDE BAR AND GALLEY,2418653.0,Restaurant,Risk 1 (High),544-546 N WELLS ST,CHICAGO,IL,60654.0,7/3/2018,Canvass Re-Inspection,Pass,NaN,41.892131,-87.634227,"(-87.63422673785976, 41.892130839277115)"
11,2184219,PATSY'S PLACE,PATSY'S PLACE,1518594.0,Restaurant,Risk 1 (High),2825 N LINCOLN AVE,CHICAGO,IL,60657.0,7/3/2018,Complaint Re-Inspection,Pass,NaN,41.933218,-87.659421,"(-87.65942136953332, 41.93321847508229)"


In [256]:
#we only need columns: Facility Type, Risk, Inspection Date, Inspection Type, and Violations. 
df = df[["Risk", "Facility Type", "Inspection Date","Inspection Type", "Violations"]]
print(df.shape)
df.head()

(12058, 5)


,Risk,Facility Type,Inspection Date,Inspection Type,Violations
2,Risk 1 (High),Restaurant,7/3/2018,Canvass Re-Inspection,NaN
3,Risk 1 (High),Restaurant,7/3/2018,Canvass Re-Inspection,NaN
8,Risk 1 (High),Restaurant,7/3/2018,Complaint Re-Inspection,NaN
10,Risk 1 (High),Restaurant,7/3/2018,Canvass Re-Inspection,NaN
11,Risk 1 (High),Restaurant,7/3/2018,Complaint Re-Inspection,NaN


In [257]:
df.count() #Violations and Facility columns have NaNs

Risk               12058
Facility Type      12048
Inspection Date    12058
Inspection Type    12058
Violations          8112
dtype: int64

In [258]:
df = df.dropna(axis = 'index', how = "any") #drop any row which contain an NaN
df.reset_index(drop = True, inplace=True) #this is the smallest number of rows, so reset indices
print(df.shape)
df.head(10)

(8105, 5)


,Risk,Facility Type,Inspection Date,Inspection Type,Violations
0,Risk 2 (Medium),Restaurant,7/3/2018,Canvass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
1,Risk 2 (Medium),Restaurant,7/6/2018,License,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...
2,Risk 2 (Medium),Grocery Store,7/9/2018,Canvass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...
3,Risk 1 (High),Restaurant,7/9/2018,Complaint,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
4,Risk 1 (High),Restaurant,7/9/2018,Canvass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
5,Risk 1 (High),Restaurant,7/10/2018,Canvass,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...
6,Risk 1 (High),Children's Services Facility,7/11/2018,License,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ..."
7,Risk 1 (High),Restaurant,7/11/2018,Canvass Re-Inspection,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...
8,Risk 1 (High),Children's Services Facility,7/11/2018,License,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ..."
9,Risk 1 (High),Restaurant,7/12/2018,License Re-Inspection,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...


In [259]:
#return count of each category in 'Facility Type' column
df['Facility Type'].value_counts().head()


Restaurant                         5205
Grocery Store                       849
School                              835
Children's Services Facility        329
Daycare Above and Under 2 Years     195
Name: Facility Type, dtype: int64

In [260]:
#subset df to only include "Restaurant" in "Facility Type" column
df = df[df['Facility Type']=='Restaurant']
df.reset_index(drop = True, inplace = True) #reset index
print(len(df))
df.head()

5205


,Risk,Facility Type,Inspection Date,Inspection Type,Violations
0,Risk 2 (Medium),Restaurant,7/3/2018,Canvass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
1,Risk 2 (Medium),Restaurant,7/6/2018,License,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...
2,Risk 1 (High),Restaurant,7/9/2018,Complaint,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
3,Risk 1 (High),Restaurant,7/9/2018,Canvass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
4,Risk 1 (High),Restaurant,7/10/2018,Canvass,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...


In [261]:
#return count of each category in 'Inspection Type' column
df['Inspection Type'].value_counts().head()

Canvass                    2614
Canvass Re-Inspection       845
Complaint                   603
License                     412
Complaint Re-Inspection     365
Name: Inspection Type, dtype: int64

In [262]:
#subset df to only include "Canvass" in "Inspection Type" column
df = df[df['Inspection Type']=='Canvass']
df.reset_index(drop = True, inplace = True) #reset index
print(len(df))
df.head()

2614


,Risk,Facility Type,Inspection Date,Inspection Type,Violations
0,Risk 2 (Medium),Restaurant,7/3/2018,Canvass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
1,Risk 1 (High),Restaurant,7/9/2018,Canvass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
2,Risk 1 (High),Restaurant,7/10/2018,Canvass,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...
3,Risk 1 (High),Restaurant,7/12/2018,Canvass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...
4,Risk 1 (High),Restaurant,7/13/2018,Canvass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...


In [263]:
#rename Risk entries to "High", "Med", and "Low"
def riskTypes(series):
    '''Updates Risk Type categorical names to a "High", "Med", or "Low" '''
    for idx, entry in enumerate(series):
        if entry == "Risk 2 (Medium)":
            series[idx] = "Med"
        if entry == "Risk 1 (High)":
            series[idx] = "High"
        if entry == "Risk 3 (Low)":
            series[idx] = "Low"
    return series

df['Risk']= riskTypes(df["Risk"])
df.head()
#this also works but is less efficient (searches each row across all columns)
#df.apply(riskTypes, axis = 'columns')

,Risk,Facility Type,Inspection Date,Inspection Type,Violations
0,Med,Restaurant,7/3/2018,Canvass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
1,High,Restaurant,7/9/2018,Canvass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
2,High,Restaurant,7/10/2018,Canvass,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...
3,High,Restaurant,7/12/2018,Canvass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...
4,High,Restaurant,7/13/2018,Canvass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...


In [267]:
#re-assign Inspection Date entries to seasons
def season(series):
    '''Updates Inspection Date column to reflect a season. 
    Inspetion Date entries are strings in M/DD/YYY format. 
    This splits the string based on "/", converts the first term to a int, 
    and then assigns a season based on the number. 
    
    Season Assignments: 
    3-5 = Spring
    6-8 = Summer
    9-11 = Fall 
    12-2 = Winter '''
    
    for idx, entry in enumerate(series):
        date_lst = entry.split('/') #['M','D','YYYY']
        if ((int(date_lst[0]) >=3) & (int(date_lst[0]) <=5)):
            series[idx] = "Spring"
        if ((int(date_lst[0]) >=6) & (int(date_lst[0]) <=8)):
            series[idx] = "Summer"
        if ((int(date_lst[0]) >=9) & (int(date_lst[0]) <=11)):
            series[idx] = "Fall"
        if ((int(date_lst[0]) ==12)| (int(date_lst[0]) ==) | (int(date_lst[0]) ==2)):
            series[idx] = "Winter"
    return series

df['Inspection Date']= season(df["Inspection Date"])
df.head()

,Risk,Facility Type,Inspection Date,Inspection Type,Violations
0,Med,Restaurant,Summer,Canvass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...
1,High,Restaurant,Summer,Canvass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
2,High,Restaurant,Summer,Canvass,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...
3,High,Restaurant,Summer,Canvass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...
4,High,Restaurant,Summer,Canvass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...


In [268]:
df.tail()

,Risk,Facility Type,Inspection Date,Inspection Type,Violations
2609,High,Restaurant,Winter,Canvass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
2610,High,Restaurant,Winter,Canvass,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...
2611,High,Restaurant,Winter,Canvass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...
2612,High,Restaurant,Winter,Canvass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ..."
2613,High,Restaurant,Winter,Canvass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...


In [269]:
df['Inspection Date'].value_counts() #checking output 

Winter    907
Fall      626
Spring    574
Summer    507
Name: Inspection Date, dtype: int64

In [273]:
#now we need to parse the violations text block into separate columns with categorical indicators 
#of whether a row is tagged with that violation (YES or NO, 1 or 0)

#create list of violation code category names
lst = ["Supervision", "EmployeeHealth", "HygenicPractices", "HandContamination", 
      "ApprovedSource", "Contamination", "TimeTemp", "ConsumerAdvisory",
       "SusceptiblePop","AdditiveSafety","ApprovedProcedures", "SafeFood", 
      "FoodTemperature", "FoodIdentification", "FoodContamination", "ProperUtensils", 
      "Equipment", "PhysicalFacilities", "EmployeeTraining", "OrdinanceCompliance"]

#create violations dataframe
vdf = pd.DataFrame(columns = lst)

#create violation dictionary lookup 
vdict = {"Supervision":[1,2], "EmployeeHealth":[3,4,5], "HygenicPractices":[6,7], "HandContamination":[8,9,10], 
      "ApprovedSource":[11,12,13,14], "Contamination":[15,16,17], "TimeTemp":[18,19,20,21,22,23,24], "ConsumerAdvisory":[25],
       "SusceptiblePop":[26],"AdditiveSafety":[27,28],"ApprovedProcedures":[29], "SafeFood":[30,31,32], 
      "FoodTemperature":[33,34,35,36], "FoodIdentification":[37], "FoodContamination":[38,39,40,41,42], "ProperUtensils":[43,44,45,46], 
      "Equipment":[47,48,49], "PhysicalFacilities":[50,51,52,53,54,55,56], "EmployeeTraining":[57,58], "OrdinanceCompliance":[59,60,61,62,63,64]}

#go through each row of the dataframe
for i in range(len(df)):
    #create row dictionary which will append to the violations_df
    row = {}
    #parse the violation string block
    string_lst = re.split("\| ", df["Violations"][i])
    code_lst = []
    #extract the violation codes
    for string in string_lst: 
        code = re.findall(r"^\d+", string)
        #finds violation descriptor given a violation code
        key = next(key for key, value in vdict.items() if int(code[0]) in value)
        #update row update
        row[key] = 1
    #append row to violations dataframe    
    vdf= vdf.append(row, ignore_index = True)

#fill NaN with 0
vdf.fillna(0, inplace = True)

#merge mini and vdf dataframe into a cleaned dataframe
cleaned_df = pd.concat([df[["Risk", "Facility Type", "Inspection Date","Inspection Type"]], vdf], axis = 1)
cleaned_df.head(5)


,Risk,Facility Type,Inspection Date,Inspection Type,Supervision,EmployeeHealth,HygenicPractices,HandContamination,ApprovedSource,Contamination,...,ApprovedProcedures,SafeFood,FoodTemperature,FoodIdentification,FoodContamination,ProperUtensils,Equipment,PhysicalFacilities,EmployeeTraining,OrdinanceCompliance
0,Med,Restaurant,Summer,Canvass,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,High,Restaurant,Summer,Canvass,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,High,Restaurant,Summer,Canvass,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3,High,Restaurant,Summer,Canvass,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,High,Restaurant,Summer,Canvass,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [275]:
#write cleaned_df to csv file
cleaned_df.to_csv("cleanedInspection.csv")

End